SQLDatabase的使用

#远程或者本地mysql

In [14]:
from langchain_community.utilities import SQLDatabase

# 测试连接本地的mysql数据库
db_user = "root"
db_password = "123456"
db_host = "localhost" #或 127.0.0.1
db_port = "3306"
db_database = "admin_account"

# mysql+pymysql://用户名:密码@ip地址:端口号/数据库名
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}")

print("操作的是哪种数据库：", db.dialect)
print("获取数据库中的表：", db.get_usable_table_names())

# 执行查询操作
res = db.run("SELECT COUNT(*) FROM common_device_config")
print(res)


操作的是哪种数据库： mysql
获取数据库中的表： ['admin_account', 'admin_auth_dept', 'admin_auth_role', 'admin_auth_role_access', 'admin_auth_rule', 'attachment', 'business_account', 'business_attachment', 'business_auth_dept', 'business_auth_role', 'business_auth_role_access', 'business_auth_rule', 'business_channel', 'business_device', 'business_device_category', 'business_home_quickop', 'business_user', 'common_api', 'common_channel_config', 'common_channel_config_19_1761025051843770900', 'common_channel_config_20_1761115259273177000', 'common_communication_config', 'common_config', 'common_device_config', 'common_device_config_19_1761025051843770900', 'common_device_config_20_1761115259273177000', 'common_dictionary_data', 'common_dictionary_group', 'common_email', 'common_generatecode', 'common_generatecode_field', 'common_message', 'common_picture', 'common_picture_cate', 'common_sys_login_log', 'common_sys_operation_log', 'common_ui_config', 'createcode_product', 'createcode_product_cate']
[(4,)]


# 本地 Sqllite

In [13]:
import sqlite3

db_path = "D:/zy/code/Testdata/LData.db"
conn = sqlite3.connect(db_path)  # 连接到 SQLite 数据库
cursor = conn.cursor()

cursor.execute("SELECT COUNT(*) FROM [020304]")
result = cursor.fetchone()
print(result)

cursor.close()
conn.close()


(105,)


# 本地大模型数据库Agent连接mysql

In [30]:
# pip install -U langchain langchain-community langchain-openai
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase

#1、获取mysql数据库的连接
# 测试连接本地的mysql数据库
db_user = "root"
db_password = "123456"
db_host = "localhost" #或 127.0.0.1
db_port = "3306"
db_database = "admin_account"

# mysql+pymysql://用户名:密码@ip地址:端口号/数据库名
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}")

# 2、获取大语言模型
import os
import dotenv
from langchain_ollama import ChatOllama

dotenv.load_dotenv()

llm = ChatOllama(
    # model = "deepseek-r1:32b",   
    model = "llama3.1:8b", 
    validate_model_on_init=True,
    reasoning=False,
    base_url="http://localhost:12356",
)

from langchain_community.agent_toolkits import SQLDatabaseToolkit

# 3、创建create_sql_query_chain的实例
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

from langchain.agents import create_agent

system_prompt = '你是一个数据库查询助手, common_device_config表是数据库中存储设备配置的表，表中字段enabled只有0和1两种状态，其中0表示设备未启用，1表示启用，表中字段device_name表示设备名称'

agent = create_agent(
    llm,
    tools,
    system_prompt=system_prompt,
)

question = "设备配置表中未启用的设备有哪些"
# question = "请告诉我有多少个表"


for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

设备配置表中未启用的设备有哪些
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (8f3e5d8d-8db1-4943-821a-7dd160a32fc2)
 Call ID: 8f3e5d8d-8db1-4943-821a-7dd160a32fc2
  Args:
    query: SELECT device_name FROM common_device_config WHERE enabled = 0
================================= Tool Message =================================
Name: sql_db_query

[('AVL415',)]
================================== Ai Message ==================================

根据数据库中的查询结果，表中未启用的设备有AVL415。


# 本地大模型数据库Agent连接Sqllite

In [35]:
# pip install -U langchain langchain-community langchain-openai
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase

import sqlite3

db_path = "D:/zy/code/Testdata/LData.db"
# 使用 SQLDatabase 而不是 sqlite3.Cursor
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

# 2、获取大语言模型
import os
import dotenv
from langchain_ollama import ChatOllama

dotenv.load_dotenv()

llm = ChatOllama(
    # model = "deepseek-r1:32b",   
    model = "llama3.1:8b", 
    validate_model_on_init=True,
    reasoning=False,
    base_url="http://localhost:12356",
)

from langchain_community.agent_toolkits import SQLDatabaseToolkit

# 3、创建create_sql_query_chain的实例
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

from langchain.agents import create_agent

system_prompt = '你是一个数据库查询助手, 【20240315_外特性_1】表是数据库中存储外特性数据的表'  # sqlite需要[]包括，以后再处理,加到提示词模板中

agent = create_agent(
    llm,
    tools,
    system_prompt=system_prompt,
)

question = "存储外特性数据的表的数据是什么，请注意，sqllite的表需要使用[]包起来进行查询,只需要显示转速字段中值大于1300和扭矩字段中大于1500的记录"
# question = "请告诉我有多少个表"


for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

存储外特性数据的表的数据是什么，请注意，sqllite的表需要使用[]包起来进行查询,只需要显示转速字段中值大于1300和扭矩字段中大于1500的记录
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (25bdd782-cc56-47c0-86c7-d8f92299e4a4)
 Call ID: 25bdd782-cc56-47c0-86c7-d8f92299e4a4
  Args:
    query: SELECT * FROM [20240315_外特性_1] WHERE [转速] > 1300 AND [扭矩] > 1500
================================= Tool Message =================================
Name: sql_db_query

[(58, '2024-03-15 11:10:44.972', 0, 'ProgRec', '', 'PowerMap循环:1/1;工况:2/2;当前程控时间:00:01:09.0;累计程控时间:00:01:09.0', 0.0, 0.0, 1747.6907958984375, 1303.3575439453125, 100.0, 39.5263671875, 238.52000427246094, 0.0, 1314.0498046875, 100.0, 0.0, 1.0, 1.0, 26.899999618530273, 27.30000114440918, 68.5999984741211, 75.9000015258789, 122.5999984741211, 46.79999923706055, 0.0, 0.0, 0.0, 0.0, 65.30000305175781, 0.0, 0.0, 0.0, 0.0, 0.0, 471.0, 28.0, 28.0, 28.0, 28.0, 28.0, 1